In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from pandas import ExcelWriter
import math
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import f1_score, accuracy_score
import os
from keras import regularizers
from numpy import array, hstack, vstack
from sklearn.utils import shuffle

from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot
 

os.chdir("C:/Users/VIP13/Статья 2/Котировки 6")

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]
 
# fit an xgboost model and make a one step prediction
def xgboost_forecast(train,eta, gamma, max_depth):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=100, eta=eta, gamma=gamma, max_depth=max_depth)
	model.fit(trainX, trainy)
	# make a one-step prediction
# 	yhat = model.predict(asarray([testX]))
	return model
 


# Functions for tbbss/tssbb and tbs/tsb

In [ ]:

import random

# tbbss/tssbb
def transf_learn(frame_learn, frame_transf, n_in, len_wind, step_smooth, eta, gamma, max_depth):
    iteratation_5=int((len(frame_learn)-len_wind)/step_smooth)+1
#     iteratation_5=15

    # prepared a dataframe to put the results of the prediction in it
    columnss=[]
    columnss.append(frame_learn.columns[0])
    for k in range(0,len(frame_transf.columns)):
        columnss.append(str(frame_transf.columns[k]))

    prog=pd.DataFrame(columns=columnss)
    orig=pd.DataFrame(columns=columnss)

    for i in range(0, iteratation_5):
    #They merged two frames together in sequence. first they beat them into samples, then they merged them
        frame_learn_use=pd.DataFrame(frame_learn.iloc[0+step_smooth*i:len_wind+step_smooth*i])
        frame_learn_use=series_to_supervised(frame_learn_use, n_in=n_in)
    
        for j in range(0,len(frame_transf.columns)):
            fr2=pd.DataFrame(frame_transf[frame_transf.columns[j]].iloc[0+step_smooth*i:len_wind+step_smooth*i])
            fr2=series_to_supervised(fr2, n_in=n_in)
            frame_learn_use=vstack([frame_learn_use, fr2])

        model = xgboost_forecast(frame_learn_use, eta=eta, gamma=gamma, max_depth=max_depth)

        # forecasting
        tmp_prog=[]
        tmp_orig=[]

        series=DataFrame(frame_learn.iloc[0+step_smooth*i:len_wind+1+step_smooth*i])
        values = series.values
        # print(series)
        data_test = series_to_supervised(values, n_in=n_in)[-1]
        testX, testy = data_test[:-1], data_test[-1]
        yhat = model.predict(asarray([testX]))[0]
        tmp_prog.append(yhat)
        tmp_orig.append(testy)

        for j in range(0,len(frame_transf.columns)):
            series=DataFrame(frame_transf.iloc[0+step_smooth*i:len_wind+1+step_smooth*i,j:j+1])
            values = series.values
            data_test = series_to_supervised(values, n_in=n_in)[-1]
            testX, testy = data_test[:-1],data_test[-1]
            yhat = model.predict(asarray([testX]))[0]
            tmp_prog.append(yhat)
            tmp_orig.append(testy)
        prog.loc[i]=np.exp(tmp_prog)
        orig.loc[i]=np.exp(tmp_orig)
    return orig, prog


# tbs/tsb
def one_learn(frame_learn, frame_forecast, n_in, len_wind, step_smooth, eta, gamma, max_depth):
    iteratation_5=int((len(frame_learn)-len_wind)/step_smooth)+1
#     iteratation_5=15

    # frame for results of forecasting
    columnss=[]
    columnss.append(frame_learn.columns[0])
    for k in range(0,len(frame_forecast.columns)):
        columnss.append(str(frame_forecast.columns[k]))

    prog=pd.DataFrame(columns=columnss)
    orig=pd.DataFrame(columns=columnss)
    for i in range(0, iteratation_5):
    #обучили
        frame_learn_use=pd.DataFrame(frame_learn.iloc[0+step_smooth*i:len_wind+step_smooth*i])
        frame_learn_use=series_to_supervised(frame_learn_use, n_in=n_in)
        model = xgboost_forecast(frame_learn_use, eta=eta, gamma=gamma, max_depth=max_depth)

        # forecasting
        tmp_prog=[]
        tmp_orig=[]

        series=DataFrame(frame_learn.iloc[0+step_smooth*i:len_wind+1+step_smooth*i])
        values = series.values
        # print(series)
        data_test = series_to_supervised(values, n_in=n_in)[-1]
        testX, testy = data_test[:-1], data_test[-1]
        yhat = model.predict(asarray([testX]))[0]
        tmp_prog.append(yhat)
        tmp_orig.append(testy)

        for j in range(0,len(frame_forecast.columns)):
            series=DataFrame(frame_forecast.iloc[0+step_smooth*i:len_wind+1+step_smooth*i,j:j+1])
            values = series.values
            data_test = series_to_supervised(values, n_in=n_in)[-1]
            testX, testy = data_test[:-1],data_test[-1]
            yhat = model.predict(asarray([testX]))[0]
            tmp_prog.append(yhat)
            tmp_orig.append(testy)
        prog.loc[i]=np.exp(tmp_prog)
        orig.loc[i]=np.exp(tmp_orig)
    return orig, prog

# Frame for results

In [ ]:
names=[]
for eta in np.arange(0.1,1,0.1):
    for gamma in np.arange(0.1,1,0.1):
        for max_depth in np.arange(1,11,1):
#             for reg_lambda in np.arange(0,1,0.1):
#                 for reg_alpha in np.arange(0,1,0.1):
#                     for subsample in np.arange(0,1,0.1):
#                         for colsample_bytree in np.arange(0,1,0.1):
            name='boost'+'_'+str(eta)+'_'+str(gamma)+'_'+str(max_depth)
            names.append(name)

all_frame_btc=pd.DataFrame(columns=names)    
all_frame_snp=pd.DataFrame(columns=names)   

# Data for TL

In [ ]:
frame_3=pd.read_csv('all_snp_btc_rv_rr.csv', delimiter=';')
frame=frame_3.iloc[:,1:].copy() #на входе подаются квадраты волатильностей, поэтому сначала из них берем корни, а потом логарифмируем
frame=frame.loc[:, [c for c in frame.columns if (frame[c]>0).all()]]
frame=np.log(np.sqrt(frame))
ind=[i for i in range(0,len(frame))]
frame.index=ind

frame_learns=pd.DataFrame()
frame_learns['rv_shk_btc']=frame['rv_shk_btc']
# frame_learn['rv_shk_btc']=frame['rv_shk_btc']

frame_transfs=pd.DataFrame()
frame_transfs['rv_shk_snp']=frame['rv_shk_snp']

# tbbss/tssbb TL

In [ ]:
frame_3=pd.read_csv('all_snp_btc_rv_rr.csv', delimiter=';')
frame=frame_3.iloc[:,1:].copy() #на входе подаются квадраты волатильностей, поэтому сначала из них берем корни, а потом логарифмируем
frame=frame.loc[:, [c for c in frame.columns if (frame[c]>0).all()]]
frame=np.log(np.sqrt(frame))
ind=[i for i in range(0,len(frame))]
frame.index=ind

frame_learns=pd.DataFrame()
frame_learns['rv_shk_btc']=frame['rv_shk_btc']
# frame_learn['rv_shk_btc']=frame['rv_shk_btc']

frame_transfs=pd.DataFrame()
frame_transfs['rv_shk_snp']=frame['rv_shk_snp']

all_frame_btc1=all_frame_btc.copy()
all_frame_snp1=all_frame_snp.copy()

for eta in np.arange(0.1,1,0.1):
    for gamma in np.arange(0.1,1,0.1):
        for max_depth in np.arange(1,11,1):
            ll=ll+1
            name='boost'+'_'+str(eta)+'_'+str(gamma)+'_'+str(max_depth)
#           tbbss
            prog=transf_learn(frame_learn=frame_learns, frame_transf=frame_transfs, 
                n_in=5, len_wind=399, step_smooth=5, eta=eta, gamma=gamma, max_depth=max_depth)
            all_frame_btc1[name]=prog[1]['rv_shk_btc']
            all_frame_snp1[name]=prog[1]['rv_shk_snp']
            print(ll)
            
            

all_frame_btc1['orig_data']=prog[0]['rv_shk_btc']
all_frame_snp1['orig_data']=prog[0]['rv_shk_snp']
all_frame_btc1.to_csv('tbbss_boosting_btc.csv')
all_frame_snp1.to_csv('tbbss_boosting_snp.csv')

# tbs TL

In [ ]:
all_frame_btc2=all_frame_btc.copy()
all_frame_snp2=all_frame_snp.copy()

for eta in np.arange(0.1,1,0.1):
    for gamma in np.arange(0.1,1,0.1):
        for max_depth in np.arange(1,11,1):
            ll=ll+1
            name='boost'+'_'+str(eta)+'_'+str(gamma)+'_'+str(max_depth)
#           tbs
            prog=one_learn(frame_learn=frame_learns, frame_forecast=frame_transfs, 
                n_in=5, len_wind=399, step_smooth=5, eta=eta, gamma=gamma, max_depth=max_depth)
            all_frame_btc2[name]=prog[1]['rv_shk_btc']
            all_frame_snp2[name]=prog[1]['rv_shk_snp']
            print(ll)
            
            

all_frame_btc2['orig_data']=prog[0]['rv_shk_btc']
all_frame_snp2['orig_data']=prog[0]['rv_shk_snp']
all_frame_btc2.to_csv('tbs_boosting_btc.csv')
all_frame_snp2.to_csv('tbs_boosting_snp.csv')

# tsb TL

In [ ]:
all_frame_btc3=all_frame_btc.copy()
all_frame_snp3=all_frame_snp.copy()

for eta in np.arange(0.1,1,0.1):
    for gamma in np.arange(0.1,1,0.1):
        for max_depth in np.arange(1,11,1):
            ll=ll+1
            name='boost'+'_'+str(eta)+'_'+str(gamma)+'_'+str(max_depth)
#            tsb
            prog=one_learn(frame_learn=frame_transfs, frame_forecast=frame_learns, 
                n_in=5, len_wind=399, step_smooth=5, eta=eta, gamma=gamma, max_depth=max_depth)
            all_frame_btc3[name]=prog[1]['rv_shk_btc']
            all_frame_snp3[name]=prog[1]['rv_shk_snp']
            print(ll)
            
            

all_frame_btc3['orig_data']=prog[0]['rv_shk_btc']
all_frame_snp3['orig_data']=prog[0]['rv_shk_snp']
all_frame_btc3.to_csv('tsb_boosting_btc.csv')
all_frame_snp3.to_csv('tsb_boosting_snp.csv')

# Analyzing the results

In [ ]:



# frame=pd.read_csv('tsb_boosting_btc.csv', delimiter=',',decimal='.')
# frame=pd.read_csv('tsb_boosting_snp.csv', delimiter=',',decimal='.')
# frame=pd.read_csv('tbs_boosting_btc.csv', delimiter=',',decimal='.')
# frame=pd.read_csv('tbs_boosting_snp.csv', delimiter=',',decimal='.')
# frame=pd.read_csv('tbbss_boosting_btc.csv', delimiter=',',decimal='.')
frame=pd.read_csv('tbbss_boosting_snp.csv', delimiter=',',decimal='.')
orig=frame['orig_data']

frae=pd.DataFrame(orig)

for i in range(1, len(frame.columns)-1):
    prognoz=frame[frame.columns[i]]
#     pri=[]
#     for j in prognoz:
#         pri.append(float(j[1:-1]))
    frae[frame.columns[i]]=prognoz  
        
mape=[]
mape.append(0)
c=frae[frae.columns[0]]
for i in range(1,len(frae.columns)):
    cc=frae[frae.columns[i]]
    mape.append((100*abs(c-cc)/c).mean())
# print(mape)
fraes=frae.transpose()
fraes['mape']=mape
frae_sort=fraes.sort_values(by='mape')
frae_sort=frae_sort.transpose()
# print(frae_sort)
print(frae_sort.iloc[-1,1:6])
frame_pr=pd.DataFrame(frae[frae.columns[0]])
accuracy=[]
accuracy.append(0)
for i in range(1,len(frae.columns)):
    frame_pr['prog']=frae[frae.columns[i]]
    fr=frame_pr.iloc[1:,0:].copy()
    ind=[i for i in range(0,len(frame_pr)-1)]
    fr.index=ind
    frr=frame_pr.iloc[:-1,0:].copy()
    frr.index=ind
    frame_diff=fr-frr
    frame_diff[frame_diff < 0] = 0
    frame_diff[frame_diff > 0] = 1

    for j in range(1,len(frame_diff.columns)):
        accuracy.append(accuracy_score(frame_diff.iloc[:,0], frame_diff.iloc[:,j]))
# print(accuracy)
fraes['accuracy']=accuracy

frae_sort_1=fraes.sort_values(by='accuracy', ascending=False)
frae_sort_1=frae_sort_1.transpose()
print(frae_sort_1.iloc[-1,0:5])

In [ ]:
name_fr=pd.DataFrame(frae_sort.iloc[-1,1:6]).transpose().columns[0]
print(frae_sort_1[name_fr].iloc[-2:])

name_fr_1=pd.DataFrame(frae_sort_1.iloc[-1,0:6]).transpose().columns[0]
print(frae_sort[name_fr_1])